In [614]:
from tkinter import YView, XView

import numpy as np
import pandas as pd
np.random.seed = 123
data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [615]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [616]:
data['Age'].fillna(data['Age'].mean(),inplace=True)
data['Cabin'].fillna('N',inplace=True)
data['Embarked'].fillna('N',inplace=True)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(),inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [617]:
data.loc[data["Sex"] == "male", "Sex_encode"] = 0
data.loc[data["Sex"] == "female", "Sex_encode"] = 1

In [618]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [619]:
Y = data['Survived']
X = data.drop('Survived', axis=1)


In [620]:
X = transform_features(X)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [621]:
import numpy as np

# 로그 변환 적용 (값이 0 이하인 경우를 방지하기 위해 작은 값 더하기)
X['Fare'] = np.log(X['Fare'] + 1)
X['Age'] = np.log(X['Age'] + 1)



In [622]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, Y, \
                                                  test_size=0.2, random_state=11)

In [623]:
def batch_sampler(data, target, batch_size):
    tot = len(data)
    total_iter = tot // batch_size
    for i in range(total_iter):
        yield (data[i * batch_size:(i + 1) * batch_size],
               target[i * batch_size:(i + 1) * batch_size])


sampler = batch_sampler(data=X_train, target=y_train, batch_size=32)
L = len(X_train) // 32
for i in range(L):
    next(sampler)
print(X_train)

     Pclass  Sex       Age  SibSp  Parch      Fare  Cabin  Embarked  \
333       3    1  2.833213      2      0  2.944439      7         3   
662       1    1  3.871201      0      0  3.280441      4         3   
382       3    1  3.496508      0      0  2.188856      7         3   
331       1    1  3.839452      0      0  3.384390      2         3   
149       2    1  3.761200      0      0  2.639057      7         3   
..      ...  ...       ...    ...    ...       ...    ...       ...   
269       1    0  3.583519      0      0  4.917301      2         3   
337       1    0  3.737670      0      0  4.908972      4         0   
91        3    1  3.044522      0      0  2.180892      7         3   
80        3    1  3.135494      0      0  2.302585      7         3   
703       3    1  3.258097      0      0  2.168105      7         2   

     Sex_encode  
333         0.0  
662         0.0  
382         0.0  
331         0.0  
149         0.0  
..          ...  
269         1.0  
337

In [624]:
import numpy as np
from tqdm import tqdm
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=400,batch_size = 32, n_features=9):
        # 초기 가중치를 무작위로 설정
        self.weights = np.random.randn(n_features)
        self.b = np.random.randn()
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.loss_history = []
        self.batch_size = batch_size

    def sigmoid(self, z):
        """Sigmoid 활성화 함수"""
        return 1.0 / (1 + np.exp(-z))

    def compute_loss(self, preds, target):
        """교차 엔트로피 손실 계산"""
        # 로그의 안정성을 위해 값들을 클리핑
        preds = np.clip(preds, 1e-9, 1 - 1e-9)  # 0과 1의 범위를 벗어나지 않도록 함
        loss = -((target * np.log(preds)) + (1 - target) * np.log(1 - preds)).mean()
        return loss


    def fit(self, X, y, batch_sampler):
        """모델 훈련 함수"""
        for epoch in tqdm(range(self.epochs)):
            sampler = batch_sampler(data=X, target=y, batch_size=self.batch_size)
            batch_loss = []
    
            for X_batch, Y_batch in sampler:
                # 순방향 전파
                preds = self.sigmoid(np.dot(X_batch, self.weights) + self.b)
    
                # 손실 계산
                loss = self.compute_loss(preds, Y_batch)
                batch_loss.append(loss)
    
                # 그래디언트 계산
                dloss_dz = preds - Y_batch
                weights_deriv = np.dot(X_batch.T, dloss_dz) / X_batch.shape[0]
                b_deriv = dloss_dz.mean()
    
                # 가중치 및 바이어스 업데이트
                self.weights -= self.learning_rate * weights_deriv
                self.b -= self.learning_rate * b_deriv
    
            # Epoch별 손실 저장
            epoch_loss = np.mean(batch_loss)
            self.loss_history.append(epoch_loss)
            
            # 에포크마다 손실 출력
            print(f"Epoch {epoch + 1}/{self.epochs} - Loss: {epoch_loss:.4f}")


    def predict(self, X):
        """모델 예측 함수"""
        linear_comb = np.dot(X, self.weights) + self.b
        return self.sigmoid(linear_comb) > 0.55

    def accuracy(self, X, y):
        """정확도 계산 함수"""
        preds = self.predict(X)
        return (preds == y).mean() * 100




In [625]:
model = LogisticRegression(learning_rate=0.005, epochs=200,batch_size=32, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


 15%|█▌        | 30/200 [00:00<00:01, 145.32it/s]

Epoch 1/200 - Loss: 1.0670
Epoch 2/200 - Loss: 0.9471
Epoch 3/200 - Loss: 0.9028
Epoch 4/200 - Loss: 0.8826
Epoch 5/200 - Loss: 0.8688
Epoch 6/200 - Loss: 0.8565
Epoch 7/200 - Loss: 0.8447
Epoch 8/200 - Loss: 0.8331
Epoch 9/200 - Loss: 0.8217
Epoch 10/200 - Loss: 0.8105
Epoch 11/200 - Loss: 0.7995
Epoch 12/200 - Loss: 0.7886
Epoch 13/200 - Loss: 0.7780
Epoch 14/200 - Loss: 0.7675
Epoch 15/200 - Loss: 0.7572
Epoch 16/200 - Loss: 0.7470
Epoch 17/200 - Loss: 0.7371
Epoch 18/200 - Loss: 0.7273
Epoch 19/200 - Loss: 0.7178
Epoch 20/200 - Loss: 0.7084
Epoch 21/200 - Loss: 0.6993
Epoch 22/200 - Loss: 0.6903
Epoch 23/200 - Loss: 0.6816
Epoch 24/200 - Loss: 0.6730
Epoch 25/200 - Loss: 0.6647
Epoch 26/200 - Loss: 0.6566
Epoch 27/200 - Loss: 0.6487
Epoch 28/200 - Loss: 0.6411
Epoch 29/200 - Loss: 0.6336
Epoch 30/200 - Loss: 0.6264


 22%|██▎       | 45/200 [00:00<00:01, 143.48it/s]

Epoch 31/200 - Loss: 0.6195
Epoch 32/200 - Loss: 0.6127
Epoch 33/200 - Loss: 0.6062
Epoch 34/200 - Loss: 0.5999
Epoch 35/200 - Loss: 0.5939
Epoch 36/200 - Loss: 0.5881
Epoch 37/200 - Loss: 0.5826
Epoch 38/200 - Loss: 0.5773
Epoch 39/200 - Loss: 0.5722
Epoch 40/200 - Loss: 0.5673
Epoch 41/200 - Loss: 0.5627
Epoch 42/200 - Loss: 0.5583
Epoch 43/200 - Loss: 0.5542
Epoch 44/200 - Loss: 0.5502
Epoch 45/200 - Loss: 0.5465


 30%|███       | 60/200 [00:00<00:00, 142.15it/s]

...

Epoch 136/200 - Loss: 0.4721
Epoch 137/200 - Loss: 0.4719
Epoch 138/200 - Loss: 0.4718
Epoch 139/200 - Loss: 0.4716
Epoch 140/200 - Loss: 0.4715
Epoch 141/200 - Loss: 0.4713
Epoch 142/200 - Loss: 0.4712
Epoch 143/200 - Loss: 0.4711
Epoch 144/200 - Loss: 0.4709
Epoch 145/200 - Loss: 0.4708
Epoch 146/200 - Loss: 0.4707
Epoch 147/200 - Loss: 0.4705
Epoch 148/200 - Loss: 0.4704
Epoch 149/200 - Loss: 0.4703
Epoch 150/200 - Loss: 0.4701
Epoch 151/200 - Loss: 0.4700
Epoch 152/200 - Loss: 0.4699
Epoch 153/200 - Loss: 0.4698
Epoch 154/200 - Loss: 0.4697
Epoch 155/200 - Loss: 0.4695
Epoch 156/200 - Loss: 0.4694
Epoch 157/200 - Loss: 0.4693
Epoch 158/200 - Loss: 0.4692
Epoch 159/200 - Loss: 0.4691
Epoch 160/200 - Loss: 0.4690
Epoch 161/200 - Loss: 0.4689
Epoch 162/200 - Loss: 0.4688
Epoch 163/200 - Loss: 0.4687
Epoch 164/200 - Loss: 0.4686
Epoch 165/200 - Loss: 0.4685


 90%|█████████ | 180/200 [00:01<00:00, 143.49it/s]

Epoch 166/200 - Loss: 0.4684
Epoch 167/200 - Loss: 0.4683
Epoch 168/200 - Loss: 0.4682
Epoch 169/200 - Loss: 0.4681
Epoch 170/200 - Loss: 0.4680
Epoch 171/200 - Loss: 0.4679
Epoch 172/200 - Loss: 0.4678
Epoch 173/200 - Loss: 0.4677
Epoch 174/200 - Loss: 0.4676
Epoch 175/200 - Loss: 0.4675
Epoch 176/200 - Loss: 0.4675
Epoch 177/200 - Loss: 0.4674
Epoch 178/200 - Loss: 0.4673
Epoch 179/200 - Loss: 0.4672
Epoch 180/200 - Loss: 0.4671


100%|██████████| 200/200 [00:01<00:00, 143.60it/s]

Epoch 181/200 - Loss: 0.4671
Epoch 182/200 - Loss: 0.4670
Epoch 183/200 - Loss: 0.4669
Epoch 184/200 - Loss: 0.4668
Epoch 185/200 - Loss: 0.4667
Epoch 186/200 - Loss: 0.4667
Epoch 187/200 - Loss: 0.4666
Epoch 188/200 - Loss: 0.4665
Epoch 189/200 - Loss: 0.4664
Epoch 190/200 - Loss: 0.4664
Epoch 191/200 - Loss: 0.4663
Epoch 192/200 - Loss: 0.4662
Epoch 193/200 - Loss: 0.4662
Epoch 194/200 - Loss: 0.4661
Epoch 195/200 - Loss: 0.4660
Epoch 196/200 - Loss: 0.4660
Epoch 197/200 - Loss: 0.4659
Epoch 198/200 - Loss: 0.4658
Epoch 199/200 - Loss: 0.4658
Epoch 200/200 - Loss: 0.4657
Test Accuracy: 84.92%


In [626]:
model = LogisticRegression(learning_rate=0.005, epochs=200,batch_size=64, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1/200 - Loss: 2.1884
Epoch 2/200 - Loss: 1.8866
Epoch 3/200 - Loss: 1.7617
Epoch 4/200 - Loss: 1.7061
Epoch 5/200 - Loss: 1.6717
Epoch 6/200 - Loss: 1.6434
Epoch 7/200 - Loss: 1.6168
Epoch 8/200 - Loss: 1.5909
Epoch 9/200 - Loss: 1.5652
Epoch 10/200 - Loss: 1.5397
Epoch 11/200 - Loss: 1.5145
Epoch 12/200 - Loss: 1.4894
Epoch 13/200 - Loss: 1.4646
Epoch 14/200 - Loss: 1.4400
Epoch 15/200 - Loss: 1.4156
Epoch 16/200 - Loss: 1.3914
Epoch 17/200 - Loss: 1.3674
Epoch 18/200 - Loss: 1.3437
Epoch 19/200 - Loss: 1.3202


 24%|██▍       | 49/200 [00:00<00:00, 250.18it/s]

Epoch 20/200 - Loss: 1.2969
Epoch 21/200 - Loss: 1.2739
Epoch 22/200 - Loss: 1.2512
Epoch 23/200 - Loss: 1.2287
Epoch 24/200 - Loss: 1.2065
Epoch 25/200 - Loss: 1.1845
Epoch 26/200 - Loss: 1.1629
Epoch 27/200 - Loss: 1.1415
Epoch 28/200 - Loss: 1.1204
Epoch 29/200 - Loss: 1.0997
Epoch 30/200 - Loss: 1.0792
Epoch 31/200 - Loss: 1.0591
Epoch 32/200 - Loss: 1.0393
Epoch 33/200 - Loss: 1.0199
Epoch 34/200 - Loss: 1.0009
Epoch 35/200 - Loss: 0.9822
Epoch 36/200 - Loss: 0.9639
Epoch 37/200 - Loss: 0.9460
Epoch 38/200 - Loss: 0.9285
Epoch 39/200 - Loss: 0.9115
Epoch 40/200 - Loss: 0.8949
Epoch 41/200 - Loss: 0.8787
Epoch 42/200 - Loss: 0.8630
Epoch 43/200 - Loss: 0.8478
Epoch 44/200 - Loss: 0.8331
Epoch 45/200 - Loss: 0.8188
Epoch 46/200 - Loss: 0.8051
Epoch 47/200 - Loss: 0.7919
Epoch 48/200 - Loss: 0.7791
Epoch 49/200 - Loss: 0.7669


 39%|███▉      | 78/200 [00:00<00:00, 267.15it/s]

...

Epoch 79/200 - Loss: 0.5840
Epoch 80/200 - Loss: 0.5817
Epoch 81/200 - Loss: 0.5795
Epoch 82/200 - Loss: 0.5774
Epoch 83/200 - Loss: 0.5754
Epoch 84/200 - Loss: 0.5735
Epoch 85/200 - Loss: 0.5717
Epoch 86/200 - Loss: 0.5700
Epoch 87/200 - Loss: 0.5683
Epoch 88/200 - Loss: 0.5667
Epoch 89/200 - Loss: 0.5652
Epoch 90/200 - Loss: 0.5638
Epoch 91/200 - Loss: 0.5624
Epoch 92/200 - Loss: 0.5610
Epoch 93/200 - Loss: 0.5598
Epoch 94/200 - Loss: 0.5585
Epoch 95/200 - Loss: 0.5573
Epoch 96/200 - Loss: 0.5562
Epoch 97/200 - Loss: 0.5551
Epoch 98/200 - Loss: 0.5540
Epoch 99/200 - Loss: 0.5530
Epoch 100/200 - Loss: 0.5520
Epoch 101/200 - Loss: 0.5510
Epoch 102/200 - Loss: 0.5500
Epoch 103/200 - Loss: 0.5491
Epoch 104/200 - Loss: 0.5482
Epoch 105/200 - Loss: 0.5474
Epoch 106/200 - Loss: 0.5465
Epoch 107/200 - Loss: 0.5457
Epoch 108/200 - Loss: 0.5449


 84%|████████▎ | 167/200 [00:00<00:00, 284.22it/s]

Epoch 109/200 - Loss: 0.5441
Epoch 110/200 - Loss: 0.5434
Epoch 111/200 - Loss: 0.5426
Epoch 112/200 - Loss: 0.5419
Epoch 113/200 - Loss: 0.5412
Epoch 114/200 - Loss: 0.5405
Epoch 115/200 - Loss: 0.5398
Epoch 116/200 - Loss: 0.5392
Epoch 117/200 - Loss: 0.5385
Epoch 118/200 - Loss: 0.5379
Epoch 119/200 - Loss: 0.5372
Epoch 120/200 - Loss: 0.5366
Epoch 121/200 - Loss: 0.5360
Epoch 122/200 - Loss: 0.5354
Epoch 123/200 - Loss: 0.5348
Epoch 124/200 - Loss: 0.5343
Epoch 125/200 - Loss: 0.5337
Epoch 126/200 - Loss: 0.5331
Epoch 127/200 - Loss: 0.5326
Epoch 128/200 - Loss: 0.5320
Epoch 129/200 - Loss: 0.5315
Epoch 130/200 - Loss: 0.5310
Epoch 131/200 - Loss: 0.5305
Epoch 132/200 - Loss: 0.5299
Epoch 133/200 - Loss: 0.5294
Epoch 134/200 - Loss: 0.5289
Epoch 135/200 - Loss: 0.5284
Epoch 136/200 - Loss: 0.5280
Epoch 137/200 - Loss: 0.5275
Epoch 138/200 - Loss: 0.5270
Epoch 139/200 - Loss: 0.5265
Epoch 140/200 - Loss: 0.5261
Epoch 141/200 - Loss: 0.5256
Epoch 142/200 - Loss: 0.5251
Epoch 143/200 

100%|██████████| 200/200 [00:00<00:00, 275.82it/s]

Epoch 168/200 - Loss: 0.5148
Epoch 169/200 - Loss: 0.5144
Epoch 170/200 - Loss: 0.5141
Epoch 171/200 - Loss: 0.5137
Epoch 172/200 - Loss: 0.5134
Epoch 173/200 - Loss: 0.5131
Epoch 174/200 - Loss: 0.5127
Epoch 175/200 - Loss: 0.5124
Epoch 176/200 - Loss: 0.5120
Epoch 177/200 - Loss: 0.5117
Epoch 178/200 - Loss: 0.5114
Epoch 179/200 - Loss: 0.5111
Epoch 180/200 - Loss: 0.5107
Epoch 181/200 - Loss: 0.5104
Epoch 182/200 - Loss: 0.5101
Epoch 183/200 - Loss: 0.5098
Epoch 184/200 - Loss: 0.5095
Epoch 185/200 - Loss: 0.5092
Epoch 186/200 - Loss: 0.5089
Epoch 187/200 - Loss: 0.5086
Epoch 188/200 - Loss: 0.5083
Epoch 189/200 - Loss: 0.5080
Epoch 190/200 - Loss: 0.5077
Epoch 191/200 - Loss: 0.5074
Epoch 192/200 - Loss: 0.5071
Epoch 193/200 - Loss: 0.5068
Epoch 194/200 - Loss: 0.5065
Epoch 195/200 - Loss: 0.5062
Epoch 196/200 - Loss: 0.5059
Epoch 197/200 - Loss: 0.5056
Epoch 198/200 - Loss: 0.5053
Epoch 199/200 - Loss: 0.5051
Epoch 200/200 - Loss: 0.5048
Test Accuracy: 81.56%
